In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import nltk
from sklearn.feature_extraction.text import CountVectorizer
# from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
# from sklearn.preprocessing import LabelEncoder


from sklearn.feature_extraction.text import TfidfVectorizer
# nltk.download('punkt')
# print(nltk.data.path)

In [2]:
DF  = pd.read_csv('afterEDAFinal.csv')

In [3]:
from keras.preprocessing.sequence import pad_sequences

X = DF['content']

Y = DF['gold_label']
Y = Y.replace({'entertainment':0,'business':1,'sports':2,'science-technology':3,'world':4})


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


X_train = X_train.astype(str)
X_test = X_test.astype(str)


tokenizer = Tokenizer(nb_words=30000, char_level=False)
tokenizer.fit_on_texts(X_train)

sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)


X_train = pad_sequences(sequences, maxlen = 300, padding ='post', truncating = 'post')
X_test = pad_sequences(test_sequences, maxlen = 300, padding ='post', truncating = 'post')

X_train = torch.tensor(X_train, dtype=torch.long)
X_test = torch.tensor(X_test,dtype=torch.long)

Y_train = torch.tensor(Y_train.values, dtype=torch.long)
Y_test = torch.tensor(Y_test.values, dtype=torch.long)


unique_labels = torch.unique(Y_train)



print(unique_labels)

C:\Users\Rayed\AppData\Local\Temp\ipykernel_7012\1443954155.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y = Y.replace({'entertainment':0,'business':1,'sports':2,'science-technology':3,'world':4})
c:\Users\Rayed\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\legacy\preprocessing\text.py:98: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn(


tensor([0, 1, 2, 3, 4])


In [4]:
import torch.nn as nn
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, num_classes):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.fc = nn.Linear(embed_size, 256)
        self.relu = nn.ReLU() 
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x, dim = 1)  
        x = self.fc(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


vocab_size = 30000
embed_size = 300
num_classes = 5
# print(num_classes)
model = TextClassifier(vocab_size, embed_size, num_classes)

In [5]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)


for epoch in range(100):  
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, Y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")



Epoch 1, Loss: 1.6194766759872437
Epoch 2, Loss: 1.707760214805603
Epoch 3, Loss: 1.5443339347839355
Epoch 4, Loss: 1.581429123878479
Epoch 5, Loss: 1.5293095111846924
Epoch 6, Loss: 1.461156964302063
Epoch 7, Loss: 1.408159613609314
Epoch 8, Loss: 1.369688630104065
Epoch 9, Loss: 1.3037055730819702
Epoch 10, Loss: 1.2408596277236938
Epoch 11, Loss: 1.182614803314209
Epoch 12, Loss: 1.1013262271881104
Epoch 13, Loss: 1.0319559574127197
Epoch 14, Loss: 0.9527732133865356
Epoch 15, Loss: 0.8664296865463257
Epoch 16, Loss: 0.7918902635574341
Epoch 17, Loss: 0.7122935056686401
Epoch 18, Loss: 0.6554489135742188
Epoch 19, Loss: 0.5849693417549133
Epoch 20, Loss: 0.5287567973136902
Epoch 21, Loss: 0.45548802614212036
Epoch 22, Loss: 0.4110856354236603
Epoch 23, Loss: 0.34647223353385925
Epoch 24, Loss: 0.3143312633037567
Epoch 25, Loss: 0.26503613591194153
Epoch 26, Loss: 0.23635460436344147
Epoch 27, Loss: 0.20014768838882446
Epoch 28, Loss: 0.17225056886672974
Epoch 29, Loss: 0.15137588977

In [6]:
# model.eval()

with torch.no_grad():
    
    outputs = model(X_test)
    _, predicted_classes = torch.max(outputs, dim=1)


y_pred = predicted_classes.cpu().numpy()
y_true = Y_test.cpu().numpy()


accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted') 
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print("Test Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred))

Test Metrics:
Accuracy: 0.9502
Precision: 0.9504
Recall: 0.9502
F1-Score: 0.9500

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        48
           1       0.97      0.90      0.94        42
           2       0.98      1.00      0.99        51
           3       0.91      0.91      0.91        45
           4       0.91      0.93      0.92        55

    accuracy                           0.95       241
   macro avg       0.95      0.95      0.95       241
weighted avg       0.95      0.95      0.95       241

